In [11]:
import json
import random
from collections import defaultdict, Counter
import textblob
from textblob import TextBlob
import spacy
from spacy import displacy
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import geniatagger
from geniatagger import GeniaTagger
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import chemdataextractor
from chemdataextractor import Document
import copy

## KB-matching Methods

In [12]:
stopwords = []
with open('stopwords.txt', 'r') as f:
    for line in f:
        stopwords.append(line.strip())

In [13]:
commonwords = []
with open('common_words.txt', 'r') as f:
    for line in f:
        commonwords.append(line.strip())

In [15]:
ontology = pd.read_csv('suzuki_ontology_final_ver.csv', header=None)
type2path = {}
for i in ontology.index:
    path = list(ontology.iloc[i].dropna())
    type2path[path[-1]] = path

In [16]:
ent2type = defaultdict(set)
with open('dict_core.txt', 'r') as f:
    for line in f:
        segments = line.strip().split('\t')
        if segments[1].lower() not in stopwords+commonwords:
            ent2type[segments[1].lower()].add(segments[0])
K = max([len(key.split(' ')) for key in ent2type])

In [17]:
for ent in ent2type:
    types = list(ent2type[ent])
    ent_lem = ' '.join([lemmatizer.lemmatize(tok) for tok in ent.lower().split('_')])
    types_lem =  [' '.join([lemmatizer.lemmatize(tok) for tok in tp.lower().split('_')]) for tp in types]  
    if len(types) == 1:
        continue                    
    elif ent_lem in types_lem:
        ent2type[ent] = set([types[types_lem.index(ent_lem)]])
    else:
        types_new = []
        paths = [type2path[tp] for tp in types]
        for k,path1 in enumerate(paths):
            flag = 0
            for l,path2 in enumerate(paths):
                if path1 == path2[:len(path1)] and k != l:
                    flag = 1
                    break
            if flag == 0:
                types_new.append(types[k])
        ent2type[ent] = set(types_new)

In [18]:
# Word lookup table only (without word and char ngrams, more effective!)
ent2concept = ent2type
word2ent = defaultdict(set)
concept2word_tf = defaultdict(lambda: defaultdict(int))
concept2word_idf = defaultdict(lambda: defaultdict(int))
concept2word_tfidf = defaultdict(lambda: defaultdict(float))
word2concept_tfidf = defaultdict(lambda: defaultdict(float))
for term in ent2type:
    words = re.sub('[^0-9a-zA-Z]+', ' ', term).split()          
    for word in words:
        word = lemmatizer.lemmatize(word.lower())
        if len(word) < 3:
            continue
        if word in stopwords:
            continue
        if word.isnumeric():
            continue
        word2ent[word].add(term)
        for tp in ent2type[term]:
            concept2word_tf[tp][word] += 1
            concept2word_idf[word][tp] += 1
            
for tp in concept2word_tf:
    for word in concept2word_tf[tp]:
        tf = concept2word_tf[tp][word]/sum(concept2word_tf[tp].values())
        idf = np.log(len(type2path)/len(concept2word_idf[word]))
        concept2word_tfidf[tp][word] = tf*idf
        word2concept_tfidf[word][tp] = tf*idf

In [19]:
data_name = 'CHEM_test'

In [20]:
data = []
with open(data_name+'.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)

In [21]:
# SM_longest: String matching: matching the longest entities
data_new = []
for sent in tqdm(data):
    sent_new = {'sentid': sent['sentid'], 'sent_tokens': sent['sent_tokens'], 'entities': []}
    i = 0
    while i < len(sent['sent_tokens']):
        for j in reversed(range(K+1)):
            text = ' '.join(sent['sent_tokens'][i:i+j])
            if text.lower() in ent2type:
                type_new = ','.join(ent2type[text.lower()])
                sent_new['entities'].append({'start':i, 'end':i+j, 'type':type_new, 'text':text})
                i = i+j-1
                break
        i +=1
    data_new.append(sent_new)

100%|██████████| 3908/3908 [00:01<00:00, 2135.16it/s]


In [ ]:
with open(data_name+'_SM_longest.json', 'w') as f:
    for sent in data_new:
        json.dump(sent, f)
        f.write('\n')

In [ ]:
# NP_genia: Noun phrase extraction (trained on WSJ, GENIA, and PennBioIE) + entity linking
tagger = GeniaTagger('../corpus/geniatagger-3.0.2/geniatagger')

In [ ]:
data_new = []
for sent in tqdm(data):
    sent_new = {'sentid': sent['sentid'], 'sent_tokens': sent['sent_tokens'], 'entities': []}
    text = ' '.join(sent['sent_tokens'])
    text_genia = tagger.parse(text)
    flag = 0
    ent = []
    for i,tok in enumerate(text_genia):
        if len(tok) < 5:
            continue
        if tok[3] == 'B-NP':
            ent.append(tok[0])
            start = i
            flag = 1
        elif tok[3] == 'I-NP':
            ent.append(tok[0])
        else:
            if flag == 1:
                end = i
                # Remove Determiners
                dets = ['the', 'a', 'an', 'this', 'that', 'these', 'those', 'my', 'your', 
                        'his', 'her', 'its', 'our', 'their', 'much', 'many', 'most', 'some', 
                        'any', 'enough', 'all', 'both', 'half', 'either', 'neither', 'each',
                        'every', 'other', 'another', 'such', 'what', 'rather', 'quite']
                if len(ent) == 0:
                    continue
                if ent[0].lower() in dets:
                    ent = ent[1:]
                    start += 1
                ent_text = ' '.join(sent['sent_tokens'][start:end])
                ent_text_new = re.sub('[^0-9a-zA-Z]+', ' ', ent_text).strip()
                # Filter out words with length < 3
                if len(ent_text_new) < 3:
                    flag = 0
                    ent = []
                    continue
                # Filter stopwords or commonwords
                if ent_text_new.lower() in stopwords+commonwords:
                    flag = 0
                    ent = []
                    continue
                # Filter out numbers
                if ent_text_new.replace(' ', '').isnumeric():
                    flag = 0
                    ent = []
                    continue
                if ent_text in ent2type:
                    tp = ','.join(ent2type[ent_text])
                else:
                    tp = 'PHRASE'
                sent_new['entities'].append({'start':start, 'end':end, 'type':tp, 'text':ent_text})
                flag = 0
                ent = []
    data_new.append(sent_new)

In [ ]:
with open(data_name+'_NP_genia.json', 'w') as f:
    for sent in data_new:
        json.dump(sent, f)
        f.write('\n')

In [ ]:
# NP_flexible: NP_genia/ChemDataExtractor/Rule + flexible entity linking with word lookup table
    
# Word lookup table only (without word and char ngrams, more effective!)
def flexbleSM(entity, type_default, term2concept, word2term):
    type_new = defaultdict(int)
    for word in re.sub('[^0-9a-zA-Z]+', ' ', entity).split():
        word = lemmatizer.lemmatize(word.lower())
        if word in word2term:
            for term in word2term[word]:
                for concept in term2concept[term]:
                    type_new[concept] += 1
    if len(type_new) == 0:
        return type_default
    else:
        types = [tp for tp in type_new if type_new[tp]==max(type_new.values())]
        types_new = []
        paths = [type2path[tp] for tp in types]
        for k,path1 in enumerate(paths):
            flag = 0
            for l,path2 in enumerate(paths):
                if path1 == path2[:len(path1)] and k != l:
                    flag = 1
                    break
            if flag == 0:
                types_new.append(types[k])
        return ','.join(types_new)
    
# Incorporate TF-IDF weighting for words in each type    
def flexbleSM(entity, type_default, term2concept, word2term, word2concept_tfidf):
    type_new = defaultdict(int)
    for word in re.sub('[^0-9a-zA-Z]+', ' ', entity).split():
        word = lemmatizer.lemmatize(word.lower())
        for concept in sorted(word2concept_tfidf[word].items(), key=lambda k:k[1], reverse=True)[:1]:
            if word2concept_tfidf[word][concept[0]] > 0.01:
                type_new[concept[0]] += 1
    if len(type_new) == 0:
        return type_default
    else:
        types = [tp for tp in type_new if type_new[tp]==max(type_new.values())]
        types_new = []
        paths = [type2path[tp] for tp in types]
        for k,path1 in enumerate(paths):
            flag = 0
            for l,path2 in enumerate(paths):
                if path1 == path2[:len(path1)] and k != l:
                    flag = 1
                    break
            if flag == 0:
                types_new.append(types[k])
        return ','.join(types_new)


data_new = []
with open(data_name+'_NP_genia.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data_new.append(doc)
        break

# Incorporate ChemDataExtractor with Genia for chemical phrase mining
data_new = []
with open(data_name+'_NP_genia.json', 'r') as f:
    for line in tqdm(f, total=len(data)):
        doc = json.loads(line)
        entities_new = []
        sent = ' '.join(doc['sent_tokens'])
        span2tok_start = {}
        span2tok_end = {}
        for tok_idx in range(len(doc['sent_tokens'])-1):
            if tok_idx == 0:
                span_idx = 0
            else:
                span_idx = len(' '.join(doc['sent_tokens'][:tok_idx]))+1
            span2tok_start[span_idx] = tok_idx
            span2tok_end[len(' '.join(doc['sent_tokens'][:tok_idx+1]))] = tok_idx+1
        doc_chemdataextractor = Document(sent)
        for mention in doc_chemdataextractor.cems:
            if mention.start in span2tok_start and mention.end in span2tok_end:
                entities_new.append({'start': span2tok_start[mention.start], 'end': span2tok_end[mention.end], 
                                     'type': 'PHRASE', 'text': mention.text})
                assert(sent[mention.start:mention.end] == ' '.join(doc['sent_tokens'][span2tok_start[mention.start]:span2tok_end[mention.end]]))
        entities_new = sorted(entities_new, key=lambda k:k['start'])
        if entities_new == []:
            data_new.append(doc)
            continue
        entities_combined = copy.deepcopy(entities_new)
        ent_idx = 0
        for i, ent in enumerate(doc['entities']):
            if ent['end'] <= entities_new[ent_idx]['start']:
                entities_combined.append(ent)
            elif ent['start'] >= entities_new[ent_idx]['end']:
                while(ent['start'] >= entities_new[ent_idx]['end'] and ent_idx<len(entities_new)-1):
                    ent_idx += 1
                if ent['start'] >= entities_new[ent_idx]['end']:
                    entities_combined.append(ent)
                else:
                    if ent['end'] <= entities_new[ent_idx]['start']:
                        entities_combined.append(ent)
        entities_combined = sorted(entities_combined, key=lambda k:k['start'])
        doc['entities'] = entities_combined
        data_new.append(doc)

for i in tqdm(range(len(data_new))):
    for j,ent in enumerate(data_new[i]['entities']):
        if ent['type'] == 'PHRASE':
            # type_new = flexbleSM(ent['text'], 'PHRASE', ent2concept, word2ent)
            type_new = flexbleSM(ent['text'], 'PHRASE', ent2concept, word2ent, word2concept_tfidf)
            data_new[i]['entities'][j]['type'] = type_new

# Generate *_NP_flexible_new.json, only do majority vote for unmatched multi-token entities
for i in tqdm(range(len(data_new))):
    for j,ent in enumerate(data_new[i]['entities']):
        if ent['type'] == 'PHRASE':
            if ent['text'].lower() in ent2type and len(ent['text'].split())==1:
                type_new = ','.join(ent2type[ent['text'].lower()])
            else:
                # type_new = flexbleSM(ent['text'], 'PHRASE', ent2concept, word2ent)
                type_new = flexbleSM(ent['text'], 'PHRASE', ent2concept, word2ent, word2concept_tfidf)
                data_new[i]['entities'][j]['type'] = type_new
                

In [ ]:
with open(data_name+'_NP_flexible', 'w') as f:
    for sent in data_new:
        json.dump(sent, f)
        f.write('\n')

In [ ]:
# resolution_freq: SM_longest resolve multi-type labels with the most frequent types
data = []
with open('CHEM_train_SM_longest.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
with open('CHEM_test_SM_longest.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
type2freq = defaultdict(int)
for sent in data:
    for ent in sent['entities']:
        tps = ent['type'].split(',')
        for tp in tps:
            type2freq[tp] += 1
            
data_new = []
with open(data_name+'_SM_longest.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data_new.append(doc)
        
for i in tqdm(range(len(data_new))):
    for j,ent in enumerate(data_new[i]['entities']):
        types = ent['type'].split(',')
        freqs = [type2freq[tp] for tp in types]
        type_new = types[freqs.index(max(freqs))]
        data_new[i]['entities'][j]['type'] = type_new    

In [ ]:
with open(data_name+'_resolution_freq.json', 'w') as f:
    for sent in data_new:
        json.dump(sent, f)
        f.write('\n')

In [ ]:
# resolution_onto: NP_flexible resolve multi-type labels with the ontology structure           
data = []
with open('CHEM_train_NP_flexible.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
with open('CHEM_test_NP_flexible.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
type2freq = defaultdict(int)
for sent in data:
    for ent in sent['entities']:
        tps = ent['type'].split(',')
        for tp in tps:
            type2freq[tp] += 1
            
data_new = []
with open(data_name+'_NP_flexible.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data_new.append(doc)
        
for i in tqdm(range(len(data_new))):
    types_single = [ent['type'] for ent in data_new[i]['entities'] if ',' not in ent['type'] and ent['type'] != 'PHRASE']
    for j,ent in enumerate(data_new[i]['entities']):
        types = ent['type'].split(',')
        if len(types) == 1:
            continue
        if len(types_single) == 0:
            freqs = [type2freq[tp] for tp in types]
            type_new = types[freqs.index(max(freqs))]
            data_new[i]['entities'][j]['type'] = type_new
        else:
            tp2dist = defaultdict(int)
            for tp in types:
                for tp_single in types_single:
                    for k in range(min(len(type2path[tp]), len(type2path[tp_single]))):
                        if type2path[tp][:k] != type2path[tp_single][:k]:
                            break
                    tp2dist[tp] += k
            types_new = [tp for tp in tp2dist if tp2dist[tp] == max(tp2dist.values())]
            freqs = [type2freq[tp] for tp in types_new]
            type_new = types_new[freqs.index(max(freqs))]
            data_new[i]['entities'][j]['type'] = type_new
            

In [ ]:
with open(data_name+'_resolution_onto.json', 'w') as f:
    for sent in data_new:
        json.dump(sent, f)
        f.write('\n')

## NER Result Visualization

In [260]:
data_name = 'CHEM_test'
dataset = []

data = []
with open(data_name+'_resolution_onto.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
dataset.append(data)
data = []
with open(data_name+'_NP_flexible.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
dataset.append(data)
data = []
with open(data_name+'_rule_chunking.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
dataset.append(data)
data = []
with open(data_name+'_NP_genia.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
dataset.append(data)
data = []
with open(data_name+'_resolution_freq.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
dataset.append(data)
data = []
with open(data_name+'_SM_longest.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        data.append(doc)
dataset.append(data)

In [ ]:
for idx in range(1000):
# for idx in range(len(dataset[0])):
    print('Sentence ID:', str(idx))
    for data in dataset:
        doc = data[idx]
        doc_display = {}
        doc_display['title'] = None
        doc_display['text'] = ' '.join(doc['sent_tokens'])
        doc_display['ents'] = []
        for ent in doc['entities']:
            start = len(' '.join(doc['sent_tokens'][:ent['start']])) 
            if start !=0:
                start += 1
            end = start + len(ent['text'])
            label = ent['type']
            if label == 'PHRASE':
                continue
            doc_display['ents'].append({"start": start, "end": end, "label": label})
        displacy.render(doc_display, style="ent", manual=True)
        print('--------------------')
    print(doc_display['text'])
    print('--------------------')

Sentence ID: 0


--------------------


--------------------


--------------------
Because of the high temperature and amount of base required for the Suzuki-Miyaura reaction , the protecting group was hydrolyzed , and the corresponding alcohol was oxidized under the reaction conditions .
--------------------
Sentence ID: 1


--------------------


...

--------------------


--------------------
The allylic substitution can be paired with oxidation to provide valuable α-amino ketones and 1,4-dicarbonyl compounds , which are not accessible by the Tsuji-Trost allylation alone .
--------------------
Sentence ID: 999


--------------------


--------------------


--------------------
While many academic groups tend to focus on simplifying natural product synthesis , RCC applies to both traditional settings and ones with more translational impact ( medicines , materials , agrochemicals , etc . ) .
--------------------


## Evaluation

In [257]:
data_true = []
types = []
sent_ids = []
with open('CHEM_test_annotations.jsonl', 'r') as f:
    for line in f:
        doc = json.loads(line)
        sent_ids.append(doc['sentid'])
        for ent in doc['entities']:
            types.append(ent['type'])
        data_true.append(doc)
types = set(types)
types.remove('PHRASE')

# Remove internal node types
type_internal = set()
for tp in type2path:
    type_internal |= set(type2path[tp][:-1])    
types -= type_internal
print(types)

{'Oxoacids', 'Biomolecules', 'Organometallic_compounds', 'Coupling_reactions', 'Aromatic_compounds', 'Ligands', 'Functional_groups'}


In [274]:
data_pred = []
with open('CHEM_test_resolution_onto.json', 'r') as f:
    for line in f:
        doc = json.loads(line)
        if doc['sentid'] not in sent_ids:
            continue
        data_pred.append(doc)

In [275]:
evaluation = {tp:{'TP':0, 'FN':0, 'FP':0} for tp in types}
evaluation['Total'] = {'TP':0, 'FN':0, 'FP':0}

In [276]:
for i in range(len(data_true)):
    entities_true = data_true[i]['entities']
    entities_pred = data_pred[i]['entities']
    for entity in entities_true:
        tp = entity['type']
        if tp not in types:
            continue
        if entity in entities_pred:
            evaluation[tp]['TP'] += 1
            evaluation['Total']['TP'] += 1
            entities_pred.remove(entity)
        else:
            evaluation[tp]['FN'] += 1
            evaluation['Total']['FN'] += 1
    for entity in entities_pred:
        tp = entity['type']
        if tp not in types:
            continue
        evaluation[tp]['FP'] += 1
        evaluation['Total']['FP'] += 1

In [277]:
for tp in evaluation:
    print(tp)
    if evaluation[tp]['TP']+evaluation[tp]['FP'] == 0:
        precision = 0
    else:
        precision = evaluation[tp]['TP']/(evaluation[tp]['TP']+evaluation[tp]['FP'])
    if evaluation[tp]['TP']+evaluation[tp]['FN'] == 0:
        recall = 0
    else:
        recall = evaluation[tp]['TP']/(evaluation[tp]['TP']+evaluation[tp]['FN'])
    if precision+recall == 0:
        f1 = 0
    else:
        f1 = 2*precision*recall/(precision+recall)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1', f1)
    print(evaluation[tp]['TP'], evaluation[tp]['FN'], evaluation[tp]['FP'])
    print('----------')

Oxoacids
Precision: 0.8823529411764706
Recall: 0.8823529411764706
F1 0.8823529411764706
15 2 2
----------
Biomolecules
Precision: 1.0
Recall: 0.17647058823529413
F1 0.3
3 14 0
----------
Organometallic_compounds
Precision: 0.0
Recall: 0.0
F1 0
0 21 4
----------
Coupling_reactions
Precision: 0.7377049180327869
Recall: 0.8181818181818182
F1 0.7758620689655172
45 10 16
----------
Aromatic_compounds
Precision: 0
Recall: 0.0
F1 0
0 51 0
----------
Ligands
Precision: 0.6428571428571429
Recall: 0.6428571428571429
F1 0.6428571428571429
9 5 5
----------
Functional_groups
Precision: 1.0
Recall: 0.037037037037037035
F1 0.07142857142857142
1 26 0
----------
Total
Precision: 0.73
Recall: 0.3613861386138614
F1 0.4834437086092716
73 129 27
----------
